In [12]:
import math
import os
import time
import numpy as np
import mxnet as mx
from mxnet import gluon, autograd
from mxnet.gluon import nn, rnn

In [13]:
ctx = mx.cpu()
num_embed = 100
num_hidden = 100
num_layers = 2
dropout = 0.2
args_batch_size = 64
args_lr = 1.0

In [14]:
net = rnn.RNN(hidden_size=num_hidden, activation='relu',num_layers= num_layers, dropout = 0.2,
                                   input_size=1000)

In [15]:
import json

with open("./data/train_data_1000.json") as json_data:
    mnist = json.load(fp=json_data)

In [16]:
#mnist = mx.test_utils.get_mnist()
batch_size = 64
num_inputs = 50
num_outputs = 2
train_data = mx.io.NDArrayIter(np.array(mnist['data']), np.array(mnist['label']),
                               batch_size, shuffle=True)
test_data = mx.io.NDArrayIter(np.array(mnist['data']), np.array(mnist['label']),
                               batch_size, shuffle=True)

In [17]:

net.collect_params().initialize(mx.init.Xavier(), ctx=ctx)
trainer = gluon.Trainer(net.collect_params(), 'sgd',
                        {'learning_rate': args_lr, 'momentum': 0, 'wd': 0})
loss = gluon.loss.SoftmaxCrossEntropyLoss()

In [18]:
def evaluate_accuracy(data_iterator, net):
    acc = mx.metric.Accuracy()
    data_iterator.reset()
    for i, batch in enumerate(data_iterator):
        data = batch.data[0].as_in_context(ctx)
        label = batch.label[0].as_in_context(ctx)
        output = net(data)
        predictions = nd.argmax(output, axis=1)
        acc.update(preds=predictions, labels=label)
    return acc.get()[1]

In [23]:
def detach(hidden):
    if isinstance(hidden, (tuple, list)):
        hidden = [i.detach() for i in hidden]
    else:
        hidden = hidden.detach()
    return hidden

In [32]:
epochs = 10
moving_loss = 0.

for e in range(epochs):
    train_data.reset()
    hidden = net.begin_state(func = mx.nd.zeros, batch_size = args_batch_size, ctx = ctx)
    for i, batch in enumerate(train_data):
        data = batch.data[0].as_in_context(ctx)
        print(len(data),data.shape,len(hidden))
        
        label = batch.label[0].as_in_context(ctx)
        print(len(label))
        hidden = detach(hidden)
        with autograd.record():
            output, hidden = net(data, hidden)
            cross_entropy = loss(output, label)
            cross_entropy.backward()
        trainer.step(data.shape[0])

        ##########################
        #  Keep a moving average of the losses
        ##########################
        if i == 0:
            moving_loss = nd.mean(cross_entropy).asscalar()
        else:
            moving_loss = .99 * moving_loss + .01 * nd.mean(cross_entropy).asscalar()

    test_accuracy = evaluate_accuracy(test_data, net)
    train_accuracy = evaluate_accuracy(train_data, net)
    print("Epoch %s. Loss: %s, Train_acc %s, Test_acc %s" %
          (e, moving_loss, train_accuracy, test_accuracy))

64 (64, 50) 1
64


MXNetError: Shape inconsistent, Provided=(100,1000), inferred shape=(100,1)